In [1]:
import pandas as pd
import os
import pyarrow
import openpyxl

import os
import pyarrow
import openpyxl
import pickle
from typing import Dict

from exgauster.utils import DataOfExgauster
from dataclasses import dataclass
from typing import Optional
import exgauster.trained_booster



C:\tools\miniconda3\envs\stal\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
joined_with_future_m1 = pd.read_parquet('m1_answers_with_future_noresampled.parquet')
joined_with_future_m3 = pd.read_parquet('m3_answers_with_future_noresampled.parquet')




In [3]:
joined_with_future_m1.index[-1]

Timestamp('2023-05-10 01:00:00')

In [4]:

joined_with_future_m1_resampled = joined_with_future_m1.resample('10S').asfreq().fillna(method='bfill').fillna(method='ffill').round(decimals=0)


In [5]:
sums = joined_with_future_m1.sum()

In [6]:
sums

Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4     0.0
Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4                  0.0
Y_ЭКСГАУСТЕР А/М №4_УЛИТА ЭКСГ. №4                              0.0
Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4             0.0
Y_ЭКСГАУСТЕР А/М №4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №4                 0.0
                                                               ... 
Y_ЭКСГАУСТЕР А/М №9_МЕТРАН-100 ДАТЧИКИ ДАВЛЕНИЯ ЭКСГ.№9         0.0
Y_ЭКСГАУСТЕР А/М №9_ТСМТ-101-010-50М-80 ТЕРМОПРЕОБР.МАСЛО       0.0
Y_ЭКСГАУСТЕР А/М №9_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.1       0.0
Y_ЭКСГАУСТЕР А/М №9_ЭКСГАУСТЕР А/М №9                           0.0
Y_ЭКСГАУСТЕР А/М №9_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №9    0.0
Length: 175, dtype: float64

In [17]:

import tqdm

m1_calculated_times = joined_with_future_m1_resampled.copy().replace(1, 0)

m1_state = {
    colname: None
    for colname in joined_with_future_m1.columns
}


In [18]:
rows_to_iteration = list(joined_with_future_m1.iterrows())

In [19]:
rows_to_iteration[-1][0]

Timestamp('2023-05-10 01:00:00')

In [20]:

for index, row in tqdm.tqdm(joined_with_future_m1_resampled.iloc[::-1].iterrows(), total=len(joined_with_future_m1_resampled)):
    for colname in joined_with_future_m1_resampled.columns:
        state = m1_state[colname]
        ceil_value = row[colname]

        if state is None:
            # Сбросить состояние в 0 если нашли аварию в ячейке
            if ceil_value == 1:
                m1_state[colname] = 0

            # TODO в новом датафрейме 0 по умолчанию уже установлены
            # m1_calculated_times.at[index, colname] = 0
        else:
            # Сбросить состояние в 0 если нашли аварию в ячейке
            if ceil_value == 1:
                m1_state[colname] = 0

                # TODO в новом датафрейме 0 по умолчанию уже установлены
                # m1_calculated_times.at[index, colname] = 0
            else:
                # Увеличить время на 10 секунд (от предыдущего шага)
                new_state = state + 10
                m1_state[colname] = new_state
                # Записать расчитанное время в ячейку датафрейма
                m1_calculated_times.at[index, colname] = new_state


100%|██████████| 4267381/4267381 [31:53<00:00, 2229.68it/s]


In [22]:
m1_calculated_times

,Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_УЛИТА ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_ЗАДВИЖКА ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_ГСМ ЭКСГ. №4,Y_ЭКСГАУСТЕР А/М №4_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№4 Т.1,...,Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНЫЙ №1,Y_ЭКСГАУСТЕР А/М №9_ЗАДВИЖКА ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№9 Т.1,Y_ЭКСГАУСТЕР А/М №9_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №9,Y_ЭКСГАУСТЕР А/М №9_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ9 ВУ1,Y_ЭКСГАУСТЕР А/М №9_МЕТРАН-100 ДАТЧИКИ ДАВЛЕНИЯ ЭКСГ.№9,Y_ЭКСГАУСТЕР А/М №9_ТСМТ-101-010-50М-80 ТЕРМОПРЕОБР.МАСЛО,Y_ЭКСГАУСТЕР А/М №9_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.1,Y_ЭКСГАУСТЕР А/М №9_ЭКСГАУСТЕР А/М №9,Y_ЭКСГАУСТЕР А/М №9_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №9
time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 03:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 03:10:10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 03:10:20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 03:10:30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 03:10:40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10 00:59:20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-10 00:59:30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-10 00:59:40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
m1_calculated_times.to_parquet('submission_3.parquet')